In [53]:
# def save_embed(path, name, embed, use_wandb=False):
#     embed_file = os.path.join(path, name)
#     pickle.dump(embed, open(embed_file, "wb"))
#     if use_wandb:
#         wandb.save(embed_file)
import os
import json
from tqdm.auto import tqdm
path = 'data/beauty/Amazon_Beauty_01_01'
domain = "Beauty" 

def load_user_pref(path, domain):
    user_pref_path = os.path.join(path)
    # Load JSON data from a file
    user_pref = json.load(open(f'{user_pref_path}/user_preference_{domain}.json', 'r'))
    return user_pref

In [54]:
from translation import InitalUserEmbedding
import argparse
from utils import *
import numpy as np
import json
import os
from functools import reduce
from easydict import EasyDict as edict

class Arguemnt:
    seed = 0
    config = "config/beauty/graph_reasoning/UPGPR.json"
    
args = Arguemnt()

with open(args.config, "r") as f:
    config = edict(json.load(f))

config.seed = args.seed
transe_config = config.TRAIN_EMBEDS
transe_config.use_user_relations = config.use_user_relations
transe_config.use_entity_relations = config.use_entity_relations

assert (
    transe_config.min_epochs <= transe_config.epochs
), "Minimum number of epochs should be lower than total number of epochs."

if config.use_wandb:
    wandb.init(
        project=config.wandb_project_name, name=config.wandb_run_name, config=config
    )

transe_config.device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

config.log_dir = "{}/{}".format(config.processed_data_dir, transe_config.name)
if not os.path.isdir(config.log_dir):
    os.makedirs(config.log_dir)

# path="./data/beauty/Amazon_Beauty_01_01"
path = config.processed_data_dir
config.seed = args.seed
config_agent = config.AGENT

if config.use_wandb:
    wandb.init(
        project=config.wandb_project_name,
        name=config.wandb_run_name,
        config=config,
    )

os.environ["CUDA_VISIBLE_DEVICES"] = config_agent.gpu
config_agent.device = torch.device("cuda:0") if torch.cuda.is_available() else "cpu"

if config_agent.early_stopping == True:
    with open("early_stopping.txt", "r") as f:
        config_agent.epochs = int(f.read())

config_agent.log_dir = config.processed_data_dir + "/" + config_agent.name

In [3]:
#### initialize new user preference
init_emb = InitalUserEmbedding(
    path=path, 
    set_name="test",
    config=config
)

user_pref = load_user_pref(path, domain)

Load embedding: data/beauty/Amazon_Beauty_01_01/test_transe_embed.pkl


In [4]:
for idx in tqdm(range(len(user_pref))):
    user_id = user_pref[str(idx)]['idx_user']
    target_item = user_pref[str(idx)]['idx_item']
    user_acc_feature = user_pref[str(idx)]['user_acc_feature']
    user_rej_feature = user_pref[str(idx)]['user_rej_feature']
    user_rej_items = user_pref[str(idx)]['user_rej_items']
    
    #### Calculate initial user preference based on features
    user_pref_emb = init_emb.translation(
        user_acc_feature=user_acc_feature, 
        user_rej_feature=user_rej_feature, 
        # user_rej_items=user_rej_items
    )
    
    user_emb = init_emb.embeds['user'][user_id]
    #### Calculate who is closer to new user with top-K
    idx_cand_user, cand_user_emb = init_emb.distance(user_pref_emb, top_k=20)
    ### Insert back to user preference dictionary
    user_pref[str(idx)]['idx_cands'] = idx_cand_user.tolist()

  0%|          | 0/201 [00:00<?, ?it/s]

In [63]:
# user_pref[idx]

{'0': {'idx_user': 5164,
  'idx_item': 2085,
  'user_acc_feature': [2078],
  'user_rej_feature': [2193, 2217, 2258, 2279],
  'user_rej_items': [4096,
   9728,
   11779,
   7687,
   7692,
   8725,
   9751,
   7709,
   2591,
   8740,
   2602,
   3132,
   11330,
   4683,
   2129,
   3666,
   4199,
   9320,
   617,
   6763,
   7280,
   11381,
   10357,
   1146,
   5244,
   6273,
   2194,
   7322,
   10396,
   5794,
   9891,
   7846,
   6310,
   3240,
   5801,
   2731,
   1709,
   5806,
   7343,
   6836,
   3764,
   9405,
   5825,
   1231,
   9424,
   1751,
   11480,
   218,
   1757,
   10986,
   7403,
   4341,
   6911,
   4864,
   4354,
   6403,
   7940,
   281,
   10017,
   1315,
   2856,
   10539,
   812,
   11565,
   6958,
   1332,
   4924,
   5954,
   10056,
   9546,
   9548,
   1358,
   1360,
   8035,
   1380,
   5991,
   3435,
   5487,
   6001,
   369,
   898,
   1931,
   10123,
   5527,
   9113,
   7587,
   6565,
   11692,
   944,
   6073,
   3525,
   10183,
   10189,
   4563,
   35

In [5]:
# user_pref

In [6]:
# user_pref_emb

In [55]:
##test_agent.py
config_agent = config.AGENT
kg_config = config.KG_ARGS
    
policy_file = config_agent.log_dir + "/tmp_policy_model_epoch_{}.ckpt".format(
    config_agent.epochs
)
path_file = config_agent.log_dir + "/policy_paths_epoch_{}.pkl".format(
    config_agent.epochs
)

train_labels = load_labels(config.processed_data_dir, "train")
test_labels = load_labels(config.processed_data_dir, "test")

train_labels = load_labels(config.processed_data_dir, "train")
test_labels = load_labels(config.processed_data_dir, "test")

dataset_name = config.processed_data_dir.split("/")[-1]

model_name = (
    "UPGPR_len_"
    + str(config_agent.max_path_len)
    + "_"
    + config.AGENT.reward
    + "_"
    + config.TRAIN_EMBEDS.cold_start_embeddings
    + "_mask_"
    + str(config.AGENT.mask_first_interaction)
    + "_max_cold_concept_"
    + str(kg_config.max_nb_cold_entities)
    + "_topk_"
    + "_".join(map(str, config_agent.topk))
    + "_draft_"
)

config_agent.result_file_dir = os.path.join(
    config_agent.result_file_dir, dataset_name, model_name, str(config.seed)
)

os.makedirs(
    config_agent.result_file_dir,
    exist_ok=True,
)

In [56]:
#def predict_paths

from __future__ import absolute_import, division, print_function
import json
from collections import Counter
import os
import argparse
from math import log
from tqdm.auto import tqdm
from easydict import EasyDict as edict
import torch
from functools import reduce
from kg_env_m import BatchKGEnvironment
from actor_critic import ActorCritic
from utils import *
import wandb

set_name="test"

print("Predicting paths...")
env = BatchKGEnvironment(
    config.processed_data_dir,
    kg_config,
    set_name=set_name,
    max_acts=config_agent.max_acts,
    max_path_len=config_agent.max_path_len,
    state_history=config_agent.state_history,
    reward_function=config_agent.reward,
    mask_first_interaction=True,
    use_pattern=config_agent.use_pattern,
)
pretrain_sd = torch.load(
    policy_file, map_location=torch.device("cpu"))
model = ActorCritic(
    env.state_dim,
    env.act_dim,
    gamma=config_agent.gamma,
    hidden_sizes=config_agent.hidden,
    modified_policy=config_agent.modified_policy,
    embed_size=env.embed_size,
).to(config_agent.device)
model_sd = model.state_dict()
model_sd.update(pretrain_sd)
model.load_state_dict(model_sd)

Predicting paths...
Orginal kg.G.keys dict_keys(['user', 'item', 'brand', 'category', 'related_product', 'word', 'feature'])
delete kg.G['feaure']
Updated kg.G.keys dict_keys(['user', 'item', 'brand', 'category', 'related_product', 'word'])
Load embedding: data/beauty/Amazon_Beauty_01_01/train_transe_embed.pkl
Key embed dict_keys(['user', 'item', 'brand', 'category', 'related_product', 'word', 'also_bought', 'also_viewed', 'bought_together', 'described', 'belong_to', 'category_of', 'mentioned', 'interested_in', 'like', 'dislike', 'purchase'])


<All keys matched successfully>

In [59]:
# only select the users that have interactions
uids = [
    key
    for key in env.kg.G["user"]
    if env.kg.G["user"][key][env.kg_args.interaction]
]
# uids

In [60]:
test_labels = load_labels(config.processed_data_dir, set_name)
test_uids = list(test_labels.keys())[:201]
len(test_uids) #18307

201

In [61]:
# len(user_pref)
user_pref_uids = list(user_pref.keys())
# user_pref_uids

In [62]:
config_agent.topk = [10, 5, 3]

In [16]:
from path_utils import batch_beam_search
# batch_size = 16
start_idx = 0
user_paths, user_probs = {}, {}
for idx in tqdm(range(len(user_pref))):
    all_paths, all_probs = [], []
    batch_uids = user_pref[str(idx)]['idx_cands']
    paths, probs = batch_beam_search(
        env,
        model,
        kg_config,
        batch_uids,
        config_agent.device,
        topk=config_agent.topk,
        policy=config_agent.modified_policy,
        user_pref_embed = user_pref_emb
    )
    all_paths.extend(paths)
    all_probs.extend(probs)
    
    user_paths[idx] = all_paths
    user_probs[idx] = all_probs
    
# predicts = {"paths": paths, "probs": probs}
# pickle.dump(predicts, open(path_file, "wb"))

# if config.use_wandb:
#     wandb.save(path_file)

  0%|          | 0/201 [00:00<?, ?it/s]

/Users/guntsv/Downloads/Github/MCR/04-USER-SIMILARITY/kg_env_m.py:579: RuntimeWarning: invalid value encountered in cast
  action_embeds[i + 1] = np.concatenate((relation_embed, node_embed))


In [17]:
# user_pref['0']['idx_cands']

In [18]:
# test_labels[9334]

In [19]:
# all_paths

In [20]:
path_file = config_agent.log_dir + "/user_pref_policy_paths_epoch_{}.pkl".format(
    config_agent.epochs
)

predicts = {"paths": user_paths, "probs": user_probs}
pickle.dump(predicts, open(path_file, "wb"))

In [21]:
# predicts['paths'][0]

In [22]:
len(predicts["paths"][0])

2805

In [23]:
##evaluate_paths
# from train_transe_model import extract_embeddings 
embeds = load_embed(config.processed_data_dir, set_name)
# embeds = extract_embeddings(config)
# embeds = load_embed(dir_path, 'train')
user_embeds = embeds["user"]
interaction_embeds = embeds[kg_config.interaction][0]
item_embeds = embeds["item"]
scores = np.dot(user_embeds + interaction_embeds, item_embeds.T)

Load embedding: data/beauty/Amazon_Beauty_01_01/test_transe_embed.pkl


In [24]:
score_user_pref = np.dot(user_pref_emb + interaction_embeds, item_embeds.T)

In [25]:
# torch.topk(torch.from_numpy(score_user_pref), k=5)

In [26]:
score_user_pref.shape

(12101,)

In [27]:
scores[5164][2085]

-0.36922777

In [28]:
score_user_pref[2085]

-2.507635121517273

In [29]:
# embeds['user'][2]

In [30]:
scores.shape

(22363, 12101)

In [31]:
# 1) Get all valid paths for each user, compute path score and path probability.
results = pickle.load(open(path_file, "rb"))
pred_paths = {uid: {} for uid in test_uids}

for idx, uid_real in enumerate(pred_paths):
    for path, probs in zip(results["paths"][idx], results["probs"][idx]):
        if path[-1][1] != "item":
            continue
        uid = path[0][2]
        # if uid not in pred_paths:
        #     continue
        pid = path[-1][2]
        ### 2) Triming item which are assosiacted with user_rej_items        
        if pid in user_rej_items: 
            continue  # Skip this item if it's in the user_rej_items list
        if pid not in pred_paths[uid_real]:
            pred_paths[uid_real][pid] = []
        # path_score = scores[uid][pid]
        path_score = score_user_pref[pid]
        path_prob = reduce(lambda x, y: x * y, probs)
        pred_paths[uid_real][pid].append((path_score, path_prob, path))
        # break
    # break

In [32]:
len(pred_paths)

201

In [33]:
# 3) Pick best path for each user-product pair, also remove pid if it is in train set.
best_pred_paths = {}
for uid in pred_paths:
    train_pids = set(train_labels.get(uid, []))  
    if len(train_pids) == 0:
        continue
    best_pred_paths[uid] = []
    for pid in pred_paths[uid]:
        if pid in train_pids:
            continue
        # Get the path with highest probability
        sorted_path = sorted(pred_paths[uid][pid], key=lambda x: x[1], reverse=True)
        best_pred_paths[uid].append(sorted_path[0])
    # break

In [34]:
# pred_paths[uid][1034]

In [35]:
dir_path=config.processed_data_dir
cold_start_users_path = os.path.join(dir_path, "cold_start_users.json")
cold_start_users = json.load(open(cold_start_users_path, "r"))
cold_start_users = set(cold_start_users["train"])

In [36]:
# cold_start_users

In [37]:
# kg_file = "data/beauty/Amazon_Beauty_01_01/test_kg.pkl"
# kg = pickle.load(open(kg_file, "rb"))
# # return kg
# best_pred_paths[5164]

In [38]:
cold_path_patterns = {}
for uid in best_pred_paths:
    if uid not in cold_start_users:
        for path in best_pred_paths[uid]:
            path_pattern = path[2]
            pattern_key = ""
            for node in path_pattern:
                pattern_key += node[0] + "_" + node[1] + "-->"
            cold_path_patterns[pattern_key] = (
                cold_path_patterns.get(pattern_key, 0) + 1
            )

print(cold_path_patterns)

{}


In [39]:
# computes the item distribution from train_labels
item_distribution = Counter()
for uid in train_labels:
    item_distribution.update(train_labels[uid])

In [40]:
path_patterns = {}
for uid in best_pred_paths:
    for path in best_pred_paths[uid]:
        path_pattern = path[2]
        pattern_key = ""
        for node in path_pattern:
            pattern_key += node[0] + "_" + node[1] + "-->"
        path_patterns[pattern_key] = path_patterns.get(pattern_key, 0) + 1

In [41]:
len(path_patterns)

7

In [42]:
path_patterns

{'self_loop_user-->interested_in_category-->category_of_item-->self_loop_item-->': 9381,
 'self_loop_user-->like_brand-->belong_to_item-->self_loop_item-->': 12776,
 'self_loop_user-->self_loop_user-->like_brand-->belong_to_item-->': 915,
 'self_loop_user-->self_loop_user-->interested_in_category-->category_of_item-->': 126,
 'self_loop_user-->interested_in_category-->self_loop_category-->category_of_item-->': 487,
 'self_loop_user-->like_brand-->self_loop_brand-->belong_to_item-->': 603,
 'self_loop_user-->like_brand-->dislike_user-->purchase_item-->': 1}

In [43]:
# 3) Compute top 10 recommended products for each user.
sort_by = "score"
pred_labels = {}
for uid in best_pred_paths:
    if sort_by == "score":
        sorted_path = sorted(
            best_pred_paths[uid], key=lambda x: (x[0], x[1]), reverse=True
        )
    elif sort_by == "prob":
        sorted_path = sorted(
            best_pred_paths[uid], key=lambda x: (x[1], x[0]), reverse=True
        )
    top_pids = [p[-1][2] for _, _, p in sorted_path]  # from largest to smallest

    pred_labels[uid] = top_pids[:10]  # change order to from smallest to largest!

In [44]:
pred_labels

{5164: [1817, 2238, 338, 610, 9080, 419, 3910, 7355, 580, 792],
 1333: [512, 3067, 11097, 3111, 1817, 2237, 338, 8780, 3620, 271],
 12080: [311, 1542, 3996, 116, 6459, 8070, 2410, 2412, 2086, 5021],
 12887: [2166, 1007, 9080, 6844, 441, 57, 573, 15, 1323, 126],
 297: [2237, 338, 610, 2096, 1316, 3183, 5382, 27, 4588, 2065],
 22289: [2658, 105, 2125, 2112, 1409, 1866, 1423, 174, 129, 213],
 6214: [2412, 19, 304, 863, 28, 2048, 9189, 1007, 2650, 7971],
 20047: [10960, 8070, 2412, 6035, 2086, 3651, 5151, 5897, 2125, 1649],
 3167: [2203, 105, 268, 1866, 5895, 8318, 926, 818, 6607, 1674],
 6045: [5070, 28, 524, 4665, 2779, 8990, 1152, 371, 141, 376],
 15355: [1305, 10960, 3146, 614, 222, 1367, 3608, 972, 1801, 2650],
 4492: [1694, 3229, 4130, 2166, 93, 1007, 140, 1718, 1362, 818],
 12566: [10960, 3146, 1694, 1624, 2086, 105, 222, 3651, 5151, 5897],
 12757: [93, 1007, 184, 371, 2238, 174, 6613, 7500, 5843, 1429],
 3012: [1817, 2238, 338, 610, 9080, 1655, 419, 3910, 7355, 11343],
 5373: [3765

In [45]:
# train_labels

In [46]:
test_labels

{5164: [2085],
 1333: [10758],
 12080: [9151],
 867: [1643, 1811],
 12887: [4919, 7614, 9578, 7623],
 3563: [9918, 2171],
 297: [6846],
 22289: [9576],
 6214: [10865],
 21576: [1284],
 378: [5630],
 20047: [3541],
 3167: [11369],
 6045: [7186],
 15355: [5486],
 4492: [10371],
 12566: [9063, 8923, 2171, 7309, 9706, 167, 8180],
 12757: [1057],
 21991: [6856],
 3012: [2339],
 5373: [4870],
 4248: [2246],
 1259: [4158],
 4598: [6424],
 7562: [503],
 14357: [7182],
 1749: [11153],
 2189: [2910],
 16766: [5845],
 18384: [12005],
 1005: [11586],
 20135: [2325],
 9841: [8407],
 12552: [10907],
 7344: [9617],
 1063: [11133],
 3645: [5603],
 10969: [3200],
 4887: [3637],
 3606: [6299],
 9079: [10772],
 20406: [1901],
 13371: [10640],
 21617: [6021],
 11703: [1430],
 17322: [9523],
 13036: [5112],
 10612: [8486],
 14620: [8374],
 22019: [4807],
 9033: [4165],
 15205: [9614],
 6756: [3266],
 18137: [7689],
 6400: [1975, 10969],
 12153: [6378],
 3648: [11632],
 18831: [5467],
 1187: [6408],
 12497:

In [47]:
# train_labels

In [48]:
from path_utils import evaluate, evaluate_validation
result_file_dir = config_agent.result_file_dir
use_wandb = False

evaluate(
    pred_labels,
    test_labels,
    train_labels,
    use_wandb,
    config.processed_data_dir,
    result_file_dir=result_file_dir,
    min_items=10,
    compute_all=True,
)

NDCG=0.000 |  Recall=0.000 | HR=0.000 | Precision=0.000 | HR@1=0.000 | HR@3=0.000 | HR@5=0.000 | Computed for all users.



(0.0, 0.0, 0.0, 0.0)

In [49]:
evaluate_validation(pred_labels, test_labels)

 HR=0.000 | Invalid users=18132



0.0

In [50]:
# Example of pred_list (list of predicted items)
pred_list = [10, 5, 3, 7, 2, 8, 1]

# Example of rel_set (set of relevant items)
rel_set = {3, 7, 8}

# Initializing variables for calculation
dcg = 0.0
hit_num = 0
hit_at_1 = 0
hit_at_3 = 0
hit_at_5 = 0

# Calculating metrics based on pred_list and rel_set
for i in range(len(pred_list)):
    if pred_list[i] in rel_set:
        dcg += 1.0 / (log(i + 2) / log(2))
        hit_num += 1
        if i < 1:
            hit_at_1 += 1
        if i < 3:
            hit_at_3 += 1
        if i < 5:
            hit_at_5 += 1

# Printing results (just an example)
print(f"DCG: {dcg}")
print(f"Hit at 1: {hit_at_1}")
print(f"Hit at 3: {hit_at_3}")
print(f"Hit at 5: {hit_at_5}")

# idcg
k = 10
idcg = 0.0
for i in range(min(len(rel_set), k)):
    idcg += 1.0 / (log(i + 2) / log(2))
ndcg = dcg / idcg
recall = hit_num / len(rel_set)
precision = hit_num / k

print('ndcg', ndcg)
print('recall', recall)
print('precision', precision)

DCG: 1.2868837451814152
Hit at 1: 0
Hit at 3: 1
Hit at 5: 2
ndcg 0.6039071644781282
recall 1.0
precision 0.3


In [51]:
# if config_agent.run_path:
#     predict_paths(
#         policy_file, 
#         path_file, config, 
#         config_agent, 
#         kg_config
#     )

In [52]:
# if config_agent.run_eval:
#     evaluate_paths(
#         config.processed_data_dir,
#         path_file,
#         train_labels,
#         test_labels,
#         kg_config,
#         config.use_wandb,
#         config_agent.result_file_dir,
#         validation=False,
#     )